In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(skf.split(messages, labels)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
    
    train_dataset = CustomDataset(train_messages, train_labels, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=NUM_LABELS)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


Fold 1
Training set label distribution: {0: 3576, 1: 28, 2: 126, 3: 27, 4: 231, 5: 130, 6: 107, 7: 35, 8: 76, 9: 73, 10: 79, 11: 194, 12: 217, 13: 32, 14: 26, 15: 28, 16: 23, 17: 260, 18: 104, 19: 51, 20: 16, 21: 39, 22: 88, 23: 180, 24: 18, 25: 56, 26: 97, 27: 55, 28: 13, 29: 15, 30: 17, 31: 90, 32: 40, 33: 36, 34: 34, 35: 13, 36: 32, 37: 24, 38: 7, 39: 116, 40: 13, 41: 9, 42: 166, 43: 18, 44: 22, 45: 10, 46: 61, 47: 595, 48: 227, 49: 78}
Validation set label distribution: {0: 398, 1: 3, 2: 14, 3: 3, 4: 26, 5: 15, 6: 11, 7: 4, 8: 9, 9: 8, 10: 9, 11: 21, 12: 24, 13: 4, 14: 3, 15: 3, 16: 2, 17: 29, 18: 12, 19: 5, 20: 2, 21: 5, 22: 9, 23: 20, 24: 2, 25: 7, 26: 10, 27: 6, 28: 2, 29: 1, 30: 2, 31: 10, 32: 5, 33: 4, 34: 3, 35: 2, 36: 3, 37: 3, 39: 13, 40: 1, 41: 2, 42: 18, 43: 2, 44: 3, 45: 1, 46: 7, 47: 66, 48: 25, 49: 9}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [03:32<00:00,  2.24it/s]


...

{'val_loss': 1.4937193196899485, 'val_accuracy': 0.6958579881656805, 'precision': 0.6898648453126051, 'recall': 0.6958579881656805, 'f1-score': 0.6796775756209472, 'macro_f1-score': 0.3911093272089602, 'balanced_accuracy': 0.3743580107715947, 'mcc': 0.5910365878674003}
Epoch 10/20


100%|██████████| 476/476 [03:25<00:00,  2.31it/s]


{'val_loss': 1.536336843135222, 'val_accuracy': 0.6757396449704142, 'precision': 0.6679954788671734, 'recall': 0.6757396449704142, 'f1-score': 0.6619149655291678, 'macro_f1-score': 0.37059236214530056, 'balanced_accuracy': 0.3737926880102362, 'mcc': 0.5751035906067328}
Epoch 11/20


100%|██████████| 476/476 [03:26<00:00,  2.30it/s]


{'val_loss': 1.5903045473233708, 'val_accuracy': 0.693491124260355, 'precision': 0.7010366979987037, 'recall': 0.693491124260355, 'f1-score': 0.681269583529977, 'macro_f1-score': 0.4002431675596152, 'balanced_accuracy': 0.4013523498743489, 'mcc': 0.5937715993729454}
Early stopping triggered at epoch 11
Final 10-fold Cross-Validation Results:
val_loss: 1.480050942305858
val_accuracy: 0.6668566097032738
precision: 0.6440956863752516
recall: 0.6668566097032738
f1-score: 0.6390863534758562
macro_f1-score: 0.32164394597309265
balanced_accuracy: 0.3239753364766163
mcc: 0.5537006676053231


In [8]:
all_fold_results

[{'val_loss': 1.5161572380751065,
  'val_accuracy': 0.6658483360611019,
  'precision': 0.6326624652410242,
  'recall': 0.6658483360611019,
  'f1-score': 0.632348548260093,
  'macro_f1-score': 0.33069925839586933,
  'balanced_accuracy': 0.32590136618242693,
  'mcc': 0.5476629114149674},
 {'val_loss': 1.5376827832287963,
  'val_accuracy': 0.6615445232466509,
  'precision': 0.6466647796154749,
  'recall': 0.6615445232466509,
  'f1-score': 0.634705781471242,
  'macro_f1-score': 0.3155930710483071,
  'balanced_accuracy': 0.321744045693039,
  'mcc': 0.5469423828406613},
 {'val_loss': 1.573207688992316,
  'val_accuracy': 0.6564223798266352,
  'precision': 0.6355351792835863,
  'recall': 0.6564223798266352,
  'f1-score': 0.6304037491219757,
  'macro_f1-score': 0.3096820629843343,
  'balanced_accuracy': 0.3083190534081974,
  'mcc': 0.5390805660420614},
 {'val_loss': 1.5012411889991844,
  'val_accuracy': 0.6726677577741408,
  'precision': 0.6475430703636744,
  'recall': 0.6726677577741408,
  'f1